## Initial Setup
In this section we'll import the requisite libraries and instantiate a number of objects and variables to configure our training job

In [2]:
import sagemaker                                        # SageMaker Python SDK
from sagemaker.pytorch import PyTorch                   # PyTorch Estimator for running pytorch training jobs
from sagemaker.debugger import TensorBoardOutputConfig  # Debugger TensorBoard config to log training metrics to TensorBoard
import boto3                                            # AWS SDK for Python
import os
import tarfile
import pandas as pd
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()   # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_key_prefix = "flan-t5-finetune-for-dialogue"  # folder within bucket where code artifact will go

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Use case and Data Exploration
For this lab we'll utilize the [DialogSum Dataset](https://github.com/cylnlp/dialogsum) which is comprised of over 13K dialogues along with human provided summaries. Our goal is to finetune a model that given a dialogue will automatically generate a summary that captures all of the salient points of the conversation

In [4]:
!wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.train.jsonl -O data/dialogsum.train.jsonl
!wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.test.jsonl -O data/dialogsum.test.jsonl

--2024-04-03 01:09:02--  https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12068822 (12M) [text/plain]
Saving to: ‘data/dialogsum.train.jsonl’

data/dialogsum.trai 100%[===================>]  11.51M  --.-KB/s    in 0.03s   

2024-04-03 01:09:03 (355 MB/s) - ‘data/dialogsum.train.jsonl’ saved [12068822/12068822]

--2024-04-03 01:09:03--  https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.test.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, aw

In [5]:
# Load the train and test data into a pandas dataframe
train_data = pd.read_json("data/dialogsum.train.jsonl", lines=True)
test_data = pd.read_json("data/dialogsum.test.jsonl", lines=True)
print ("Train data shape: ", train_data.shape)
print ("Test data shape: ", test_data.shape)

Train data shape:  (12460, 4)
Test data shape:  (500, 8)


Let's take a look at a few examples

In [6]:
train_data.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [7]:
print("#####DIALOGUE###### \n", train_data["dialogue"][0])
print("\n#####SUMMARY###### \n", train_data["summary"][0])

#####DIALOGUE###### 
 #Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.

#####SUMMARY###### 
 Mr. Smith's getting a ch

In [8]:
# Upload the data to S3
# We will only use the training data which we will split into train and validation sets inside the training script
# We will use the test data to evaluate the model after we deploy it
s3_data_path = sess.upload_data("data/dialogsum.train.jsonl", bucket=bucket, key_prefix=f"{s3_key_prefix}/data")

The `accelerate launch` command has two key parts, the `config.yml` file and the `train.py` script. The `config.yml` file is used to configure the distributed training job. The `train.py` script is the training script that will be launched by the launcher. In this example, we'll use the [ds_zero3.yml](src/train/ds_zero3.yaml) configuration file. The config file enables [DeepSpeed ZeRo Stage3](#https://www.deepspeed.ai/tutorials/zero/) and a number of other optimizations to enable training of large scale models. This file was generated by running `accelerate config --config_file ds_zero3.yml` and then following the on-screen prompts. 
The [train.py](src/train/train.py) makes use of a number of key libraries to enable training of large models with minimal code changes:
- 🤗 [Accelerate](https://huggingface.co/docs/accelerate/index) - Configures the distributed training environment and adapts training objects (data loaders, models, optimizers) to the distributed environment
- 🤗 [Transformers](https://huggingface.co/docs/transformers/index) - Provides a number of pre-trained models and utilities for training and evaluating models
- 🤗 [PEFT](https://github.com/huggingface/peft) - Provides a number of methods for Parameter Efficient Finetuning(PEFT) of large language models. The [LoRA](https://arxiv.org/pdf/2106.09685.pdf) method will be used to finetune the model
- [DeepSpeed](https://github.com/microsoft/DeepSpeed) - Provides a number of optimizations to enable training of large models. In this example, we'll use DeepSpeed ZeRO Stage3 to enable training of models with over 1B parameters

In [9]:

# configure the tesnorboard output directly to S3
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f"s3://{bucket}/{s3_key_prefix}/tensorboard"
)

estimator = PyTorch(
    source_dir = "src/train",
    entry_point="acc_launcher.py",
    role=role,
    instance_count=1, 
    instance_type="ml.p4d.24xlarge", 
    framework_version="1.13",
    py_version="py39",
    disable_profiler=True,
    tensorboard_output_config=tensorboard_output_config,
    hyperparameters={"training_script": "train.py",
                     "config_file": "ds_zero3.yaml",
                     "lr": 3e-3,
                     "batch_size": 2,
                     "subsample": 50, # percent of data to use
                     "num_epochs": 2,
                     "pretrained_model_name_or_path": "google/flan-t5-xl"
                     
    },
    keep_alive_period_in_seconds=3600
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [10]:
estimator.fit({"train": s3_data_path}, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-04-03-01-09-08-479


Using provided s3_resource
2024-04-03 01:09:09 Starting - Starting the training job
2024-04-03 01:09:09 Pending - Training job waiting for capacity........................
2024-04-03 01:12:45 Pending - Preparing the instances for training........................
2024-04-03 01:16:58 Downloading - Downloading input data...
2024-04-03 01:17:33 Downloading - Downloading the training image............
2024-04-03 01:19:24 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-03 01:20:17,799 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-03 01:20:17,902 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-03 01:20:17,911 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-03 01:20:17,914 sagemaker_pytorch_container.trai

In [ ]:
from IPython.display import HTML
cur_dir = os.getcwd().replace(os.environ["HOME"],"")
HTML(f'''1. Paste the following command into the Studio Terminal <code style="background-color:gray;">tensorboard --logdir {tensorboard_output_config.s3_output_path}</code><br>
2. Click <a href='/jupyter/default/proxy/6006/'>here</a> to open TensorBoard''')

In [ ]:
# We have to wait for the job to finish before we can deploy the model 
estimator.latest_training_job.wait(logs="None")

Once the training job has completed, we can deploy the model to a SageMaker Endpoint.
We will use a [Deep learning container for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html) for deployment which is optimized for serving large models in excess of 100B parameters

In [ ]:
# We'll need a few additional imports for model deployment
from sagemaker.model import Model
from sagemaker import serializers, deserializers

In [ ]:
# This is the docker image that will be used for inference
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

The next step is to create a model deploymnent packages which will be used to deploy our model to a SageMaker Endpoint. The model deployment package is a tarball that contains the model artifacts, [inference code](src/inference/model.py), and any [additional dependencies](src/inference/requirements.txt) required to run the inference code. We'll go through the following steps to create the model deployment package:
1. Download the trained model artifact from S3 to the local filesystem
2. Cretae a `serving.properties` file that will configure our hosting environment
3. Combine the trained model, the inference code, and the `serving.properties` file into a tarball with the following structure:
```
|-- model.py         # inference code
|-- requirements.txt    # additional dependencies
|-- serving.properties  # configuration file
|-- <model_id>\         # model artifacts
    |-- config.json
    |-- pytorch_model.bin
    |-- special_tokens_map.json
    |-- tokenizer_config.json
    |-- tokenizer.json
    |-- vocab.json
```

In [ ]:
!aws s3 cp {estimator.model_data} .

Note that the model was trained using Low Rank Adaptation (LoRA), and as a result the model artifact is small (~10Mb) allowing us to repackage it along with our inference code. At deployment time, the base model will be downloaded from Hugging Face Hub and the LoRA weights will be applied to the base model. For deployment of larger models with LoRA weights, it is recommended to store the based model weights in your own S3 bucket.

In [ ]:
# extract the model artifacts into the inference code directory 
with tarfile.open("model.tar.gz", "r:gz") as tar:
    contents = tar.getnames()
    model_id = os.path.dirname(contents[-1]) # model id is the name of the folder containing the model files as generated by the training job
    tar.extractall("src/inference/")

In [ ]:
# generate the serving.properties file
# We'll use the python engine for inference and specify the model_id for the base model we want to use
with open("src/inference/serving.properties", "w") as f:
    f.write(
f"""engine=Python
option.model_id={model_id}
    """
    )

Now we have everything needed to create the model package. We'll combine the contents of the `src/inference` directory with the model artifact and create a tarball. We'll then upload the tarball to S3 and use the S3 URI to deploy the model.

In [ ]:
%%bash
cd src/
tar czvf model.tar.gz inference/
mv model.tar.gz ../

In [ ]:
hf_s3_code_artifact = sess.upload_data("model.tar.gz", bucket, f"{s3_key_prefix}/model")
print(f"S3 Code or Model tar ball uploaded to --- > {hf_s3_code_artifact}")

In [ ]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(image_uri=image_uri, model_data=model_data, role=role)

    model.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name=endpoint_name)

    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),        # will convert python dict to json
        deserializer=deserializers.JSONDeserializer(),  # will convert json to python dict
    )

    return predictor

In [ ]:
# create a unique endpoint name
hf_endpoint_name = sagemaker.utils.name_from_base("t5-summarization")
print(f"Our endpoint will be called {hf_endpoint_name}")

In [ ]:
# deployment will take 5 to 10 minutes
hf_predictor = deploy_model(
    image_uri=inference_image_uri,
    model_data=hf_s3_code_artifact,
    role=role,
    endpoint_name=hf_endpoint_name,
    instance_type="ml.g4dn.xlarge",
    sagemaker_session=sess,
)

With the endpoint deployed, we can generate summaries on dialogues from the test dataset. We'll randomly select an examples and generate summaries. You can also provide your own dialogue to generate summaries just be sure to use the same format as the examples in the train dataset 

In [ ]:
from random import randint

In [ ]:
random_dialogue_idx = randint(0, test_data.shape[0])
random_dialogue = test_data["dialogue"][random_dialogue_idx]

output = hf_predictor.predict({"inputs": [random_dialogue], "parameters":{"max_length": 100}})
output_summary = output["outputs"][0]["summary_text"]

print("#####DIALOGUE######\n", random_dialogue)
print("\n#####GENERATED SUMMARY######\n", output_summary)

In [ ]:
# delete the endpoint when finished experimenting
hf_predictor.delete_endpoint()